# Common Joins

As SQL Server is a Relational Database Management System and uses the SQL language, it makes sense that the language provides the ability to JOIN tables together.

Without the ability to JOIN tables, a Relational Database would be a redundant concept.

So now that we know the best methods of joining our tables (via Primary and Foreign Keys) and how to find those (using Database Diagrams), SQL provides several methods of JOIN that we can use, each with its own specific purpose.

Traditionally these are visualised using Venn diagrams (two intersecting circles), but I like to use a more 'table based' image instead.  For the sake of this explanation we will utilise both methods to ensure clarification whichever method you prefer.

There are two new tables we will be using to demonstrate our joins, these are subsets of two tables we have used already and called:
* hr.employeeSmall
* sales.salesOrderHeaderSmall

These tables look as follows:

In [ ]:
select *
from hr.employeeSmall;

In [ ]:
select *
from sales.salesOrderHeaderSmall;

We will be using them to create queries in which we answer the following:

-   Report One
    -   Full Name of any employee who has not made any sales
-   Report Two
    -   In what year was the oldest employee to make sales born?
-   Report Three
    -   What is the total value of sales for which we have no employee information present

#### INNER JOIN

An Inner Join will return records from both participating tables as long as there is a match between the columns of the join condition.

For example, if we join two tables on <code>a.employeeID = b.employeeID</code> then an Inner Join will only return rows where the same employeeID appears in both tables.

In Venn and tabular forms it is displayed as follows:

![InnerJoin](https://www.sqlworld.co.uk/images/Blogs/SQLServer/InnerJoin.jpg)

In terms of syntax, let's start by writing a simple query to obtain employeeID, first and last names from our employee table, remembering to use a table alias as we are preparing to join to another table:

In [ ]:
select e.employeeID, e.firstName, e.lastName
from JupyterDatabase.hr.employeeSmall as e;

Next we need to JOIN to our new table and specify which columns we are joining ON.  We shall return the employeeID and orderDate from our joined table:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
inner join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

As you can see, we have now joined the two tables together, only returning records where the employeeIDs existed in BOTH tables.

You can also see the benefits of a table alias when joining tables together, and the clarification we provided when obtaining employeeID from both tables.

It is also useful to note that in T-SQL the keyword INNER is optional within an INNER JOIN.  Simply writing JOIN will always default to an INNER JOIN.

Note that sometimes we do need to JOIN ON more than one column and this is fully supported using AND to specify additional criteria:

<code>FROM tableA as a<br>
JOIN tableB as b<br>
ON a.col1 = b.col1<br>
AND a.col2 = b.col2<br>
AND a.col3 = b.col3</code>

#### LEFT OUTER JOIN

Sometimes we wish to see more than just the matching records and this is where LEFT and RIGHT joins come into play.

A LEFT JOIN (note that the word 'outer' is optional in T-SQL) from tableA to tableB returns the entire of tableA but only those records in tableB where there is a match and is represented as follows:

![LeftJoin](https://www.sqlworld.co.uk/images/Blogs/SQLServer/LeftJoin.jpg)

A critical thing to notice from the tabular representation above is that in the result we have white space where Table 2 has no records.  These will be returned as NULL values.

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

We can see in the query results that we have the entire of our hr.employeeSmall table but only records from sales.salesOrderHeaderSmall where matches occur.  Where there are no matches (for example Kamal and Azalia), the query returns NULL values.

A common use for the LEFT JOIN is to identify records in the join table which are not present in the main table.  Ie. We wish to join two tables and return those in the FROM table where the JOIN table has NULL values.

#### RIGHT OUTER JOIN

This is the reverse of the LEFT JOIN.  In this scenario we display all records from the JOINed table and only show those in the FROM table where a match is present.  If there is no match then a NULL value is displayed.

Graphically it looks as per the following diagrams:

![RightJoin](https://www.sqlworld.co.uk/images/Blogs/SQLServer/RightJoin.jpg)

Syntactically it is the same as the LEFT JOIN and also has the word 'outer' as an optional keyword in T-SQL:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
right join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

As can clearly be seen in the results above, we have all orders returned and only employee information where present in the employee table.

Our hr.employeeSmall table does not have any information for employeeID 45 and therefore we see NULL values.

#### FULL OUTER JOIN

The FULL JOIN (again, not the omission of 'outer' in T-SQL) actually performs the fullest of all the joins as it returns both tables in their entirety.

The graphics below show that our results have every record of the FROM table, every record of the JOIN table and any records not matching are marked as NULL values:

![FullJoin](https://www.sqlworld.co.uk/images/Blogs/SQLServer/FullJoin.jpg)

This is visually similar to a LEFT JOIN and a RIGHT JOIN performed at the same time.

We can see this clearly in the output of the query below:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
full join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

#### Summary

Prior to our explanation of joins we were given a series of reports to create which we can revisit now:

-   Report One
    -   Full Name of any employee who has not made any sales
-   Report Two
    -   In what year was the oldest employee to make sales born?
-   Report Three
    -   What is the total value of sales for which we have no employee information present

Let's consider each in turn.

<u>Report One</u>

We require the full name of any employee who has not made any sales.

This can be performed by a LEFT JOIN.  A LEFT JOIN returns all values from the FROM table and NULL values from the JOIN where they are not matched.

In this case we can perform a LEFT JOIN and then ask SQL Server to return only those records which are NULL for the JOIN table.

To clarify, let's revisit our LEFT JOIN example above:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

Here we can see that we have two NULL values from the Sales table.  This is exactly what we are looking for (employees with no sales).

Let's filter our query using a WHERE clause:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID
where s.employeeID is null;

Finally we need to fulfil the 'full name' criteria, which we have covered before:

In [ ]:
select concat_ws(' ', e.title, e.firstName, e.lastName) as fullName
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID
where s.employeeID is null;

And our report is complete.

<u>Report Two</u>

In what year was the oldest employee to make sales born?

This implies that we need only consider employees who have made sales.  Therefore the employee must be in both tables and therefore we are looking for an INNER JOIN.  So we'll start there:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
inner join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

This gives us all those employees who have sales.

Next we need to determine the employee with the minimum date of birth (smalleat year being the oldest person).

Let's look back to our aggregations in order to achieve this by performing a simple table level aggregate:

In [ ]:
select min(year(dob)) as minBirthYear
from JupyterDatabase.hr.employeeSmall as e
inner join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

And we have our answer.

<u>Report Three</u>

What is the total value of sales for which we have no employee information present?

We can immediately lean towards LEFT or RIGHT join because we are interested in a situation in which we have employees not present in the sales.salesOrderHeaderSmall table.

At this stage it is worth mentioning that it does not matter which order you place the tables when joining as SQL Server will rearrange as it sees fit, therefore these are exactly the same:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall s
on e.employeeID = s.employeeID;

select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.sales.salesOrderHeaderSmall s
right join JupyterDatabase.hr.employeeSmall as e
on e.employeeID = s.employeeID;

The preference comes down to personal choice as to whether you wish to use LEFT or RIGHT and then order your tables accordingly.

As we fulfilled Report One using a LEFT JOIN we will utilise the RIGHT JOIN for this one, simply to provide more variety:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
right join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

In the above we can see the NULL values highlighting where we have sales and no employee information.

We can therefore filter to only show those records by stating we wish only where the employeeID in the hr.employeeSmall table IS NULL.

Additionally, we need to SUM the Sales Amounts for these records:

In [ ]:
select sum(s.totalDue) salesAmount
from JupyterDatabase.hr.employeeSmall as e
right join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID
where e.employeeID is null;

Now we have our three queries to support our three reports.